In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split

#Librerias para visualizacion
import seaborn as sb
import matplotlib.pyplot as plt

In [2]:
labels_training = pd.read_csv('/home/lucas/Documentos/Facultad/OrgaDeDatos/tp/labels_training_set.csv',
                              low_memory= False)
events_data = pd.read_csv('/home/lucas/Documentos/Facultad/OrgaDeDatos/tp/events_up_to_01062018.csv', 
                              low_memory= False)

In [4]:
#Data Frame de features
features = pd.DataFrame()

In [5]:
events_data.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
labels_training.head()

,person,label
0,0566e9c1,0
1,6ec7ee77,0
2,abe7a2fb,0
3,34728364,0
4,87ed62de,0


In [36]:
#En features tengo los usuarios sin repetir
features['person'] = events_data['person'].unique()

In [37]:
events_data['hour'] = pd.to_datetime(events_data['timestamp']).dt.hour
events_data['date'] = pd.to_datetime(events_data['timestamp']).dt.date

In [65]:
#cantidad de conversiones por usuario
conversions_user = pd.DataFrame()

In [66]:
conversions_user = events_data.loc[events_data['event'] == 'conversion',['person']]

In [67]:

conversions_user= pd.DataFrame(conversions_user['person'].value_counts())

In [68]:
conversions_user = conversions_user.reset_index()
conversions_user.rename({'index':'person', 'person':'cantidad de conversiones'}, axis = 1, inplace = True)
conversions_user.head()

,person,cantidad de conversiones
0,5059f7fd,129
1,888a11a2,44
2,0280a93d,32
3,b4cdf2b5,30
4,32c21f9e,21


In [69]:
features = features.merge(conversions_user, on = 'person', how = 'left')

In [70]:
features.head()

,person,cantidad de conversiones
0,4886f805,NaN
1,ad93850f,NaN
2,0297fc1e,NaN
3,2d681dd8,NaN
4,cccea85e,NaN


In [75]:
features['cantidad de conversiones'] = features['cantidad de conversiones'].fillna(0) 

In [76]:
features.head(20)

,person,cantidad de conversiones
0,4886f805,0.0
1,ad93850f,0.0
2,0297fc1e,0.0
3,2d681dd8,0.0
4,cccea85e,0.0
5,4c8a8b93,0.0
6,1b9f7cf6,0.0
7,29ebb414,0.0
8,de8fe91b,0.0
9,45baf068,0.0
